In [1]:
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.types import StructType,StructField, StringType, DateType, IntegerType, DoubleType
from pyspark.sql.functions import col, explode, array, struct, expr, sum, lit
from pyspark.sql.window import Window

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1609317990941_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
folder_s0 = 's3://dh-ud-dend-capstone/stage0_raw/'
folder_s1 = 's3://dh-ud-dend-capstone/stage1_prepared_and_partitioned/'
folder_s2 = 's3://dh-ud-dend-capstone/stage2_joined_and_partitioned/'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Create Dimensions

## dimRegion

In [3]:
df_index = spark.read.format('csv').options(header='true').load(folder_s0 + 'google/index.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df_dim_region = df_index.select("key",\
                                "country_code",\
                                "country_name",\
                                "subregion1_code",\
                                "subregion1_name",\
                                "subregion2_code",\
                                "subregion2_name",\
                                "aggregation_level")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df_dim_region = df_dim_region.withColumnRenamed("key","id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df_dim_region.write.mode('overwrite').parquet(folder_s2+'dimRegion.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# dimTime

In [7]:
# source: https://stackoverflow.com/questions/43141671/sparksql-on-pyspark-how-to-generate-time-series
df_dim_time = spark.sql("""SELECT sequence(
                                    to_date('2019-10-01'),
                                    now(), 
                            interval 1 day) as date""")\
                        .withColumn("date", explode(col("date")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
df_dim_time.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

457

In [9]:
df_dim_time = df_dim_time.withColumn("month",F.month("date"))\
           .withColumn("year",F.year("date"))\
           .withColumn("dayofmonth",F.dayofmonth("date"))\
           .withColumn("dayofyear",F.dayofyear("date"))\
           .withColumn("weekofyear",F.weekofyear("date"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
df_dim_time.write.mode('overwrite').parquet(folder_s2+'dimTime.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## dimSymptom

In [11]:
spark.sql("""CREATE TEMPORARY VIEW SEARCHTRENDS 
             USING parquet 
             OPTIONS (path \"s3://dh-ud-dend-capstone/stage1_prepared_and_partitioned/google/searchtrends.parquet\")""")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [12]:
df_dim_symptoms = spark.sql("SELECT DISTINCT symptom FROM SEARCHTRENDS")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
def determine_symptom_group(s):
    
    #source: https://www.cdc.gov/coronavirus/2019-ncov/symptoms-testing/symptoms.html
    #source: https://www.frontiersin.org/articles/10.3389/fpsyg.2020.01491/full
    
    covid_symptoms = ['fever', 'chills', 'cough', \
                  'shortness_of_breath', 'shallow_breathing', \
                  'fatigue','headache','sore_throat','throat_irritation',\
                  'nasal_congestion','nausea', 'vomiting', 'diarrhea']
    
    lockdown_symptoms = ['sleep_deprivation', 'sleep_disorder', 'anxiety', \
                  'generalized_anxiety_disorder', 'panic_attack', 'depression', \
                  'major_depressive_disorder']
    
    if s in covid_symptoms:
        return 'covid'
    elif s in lockdown_symptoms:
        return 'lockdown'
    else:
        return 'other'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# call count to materialize the dataframe
df_dim_symptoms.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

422

In [15]:
symptgrpFunc = F.udf(determine_symptom_group, StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_dim_symptoms = df_dim_symptoms.withColumn("symptom_group",symptgrpFunc(col('symptom')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df_dim_symptoms = df_dim_symptoms.select(F.row_number()\
                                     .over(Window.partitionBy()\
                                     .orderBy(df_dim_symptoms['symptom']))\
                                     .alias("id"), "symptom", "symptom_group")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
df_dim_symptoms.write.mode('overwrite').parquet(folder_s2+'dimSymptom.parquet')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Create Facts

## factSearches

In [19]:
df_st = spark.read.parquet(folder_s1+"google/searchtrends.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df_st.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- key: string (nullable = true)
 |-- date: date (nullable = true)
 |-- symptom: string (nullable = true)
 |-- search_trend: double (nullable = true)
 |-- country_code: string (nullable = true)

In [21]:
# rename key to regionId
df_st = df_st.withColumnRenamed("key", "regionId")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# join with dimsymptom to get id
df_st = df_st.join(df_dim_symptoms.select("id","symptom"),on=["symptom"],how="inner")
df_st = df_st.withColumnRenamed("id", "symptomId")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# generate row number
df_st = df_st.select(F.row_number()\
                     .over(Window.partitionBy()\
                     .orderBy(df_st['date'],df_st['regionId']))\
                     .alias("id"), "regionId", "date", "symptomId", "search_trend", "country_code")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_st.write.mode('overwrite').partitionBy("country_code").parquet(folder_s2+"/factSearches.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## factCovid

In [ ]:
#read epidemiology data
df_epi = spark.read.parquet(folder_s1+"google/epidemiology.parquet")
df_epi = df_epi.select('key', 'date', 'new_confirmed', 'new_deceased', 'new_recovered', 'new_tested')
df_epi = df_epi.withColumnRenamed('key','regionId')

In [ ]:
#read mobility data
df_mob = spark.read.parquet(folder_s1+"google/mobility.parquet")
df_mob = df_mob.select('key', 'date', 'mobility_retail_and_recreation', \
                       'mobility_grocery_and_pharmacy', 'mobility_parks', \
                       'mobility_transit_stations','mobility_workplaces',\
                       'mobility_residential')
df_mob = df_mob.withColumnRenamed('key','regionId')

In [ ]:
df_mob.printSchema()

In [ ]:
#create union of dimRegion and dimTime
df_fact_covid = df_dim_region.crossJoin(df_dim_time).select('id','date','country_code')

In [ ]:
factCount = df_fact_covid.count()

In [ ]:
df_fact_covid = df_fact_covid.withColumnRenamed('id','regionId')

In [ ]:
df_fact_covid = df_fact_covid.join(df_epi, on=['regionId','date'], how='left')

In [ ]:
df_fact_covid = df_fact_covid.join(df_mob, on=['regionId','date'], how='left')

### Oxford government response

In [ ]:
df_oxf = spark.read.parquet(folder_s1+"oxford-gov-response/oxfgovresp.parquet")

In [ ]:
# limit to NAT_TOTAL
df_oxf_f = df_oxf.filter("Jurisdiction == 'NAT_TOTAL'")

In [ ]:
df_oxf_f = df_oxf_f.select('country_code', 'date', "C1_School_closing",\
                 "C1_Flag",\
                 "C2_Workplace_closing",\
                 "C2_Flag",\
                 "C3_Cancel_public_events",\
                 "C3_Flag",\
                 "C4_Restrictions_on_gatherings",\
                 "C4_Flag",\
                 "C5_Close_public_transport",\
                 "C5_Flag",\
                 "C6_Stay_at_home_requirements",\
                 "C6_Flag",\
                 "C7_Restrictions_on_internal_movement",\
                 "C7_Flag",\
                 "C8_International_travel_controls",\
                 "E1_Income_support",\
                 "E1_Flag",\
                 "E2_Debt_contract_relief",\
                 "E3_Fiscal_measures",\
                 "E4_International_support",\
                 "H1_Public_information_campaigns",\
                 "H1_Flag",\
                 "H2_Testing_policy",\
                 "H3_Contact_tracing",\
                 "H4_Emergency_investment_in_healthcare",\
                 "H5_Investment_in_vaccines",\
                 "H6_Facial_Coverings",\
                 "H6_Flag",\
                 "H7_Vaccination_policy",\
                 "H7_Flag",\
                 "StringencyIndex",\
                 "GovernmentResponseIndex",\
                 "ContainmentHealthIndex",\
                 "EconomicSupportIndex")

In [ ]:
df_oxf_f = df_oxf_f.withColumnRenamed("country_code","regionId")

In [ ]:
df_fact_covid = df_fact_covid.join(df_oxf_f, on=['regionId','date'], how='left')

In [ ]:
factCount_final = df_fact_covid.count()

In [ ]:
assert factCount == factCount_final

In [ ]:
len(df_fact_covid.columns)

### gdelt

In [ ]:
df_gdelt = spark.read.parquet(folder_s1+"gdelt/gdelt.parquet")

In [ ]:
df_gdelt.createOrReplaceTempView("gdelt")

In [ ]:
df_gdelt_facts_covid = spark.sql("""SELECT country_code, `date`, 
                        COUNT(DISTINCT GLOBALEVENTID) as gd_events_covid,
                        SUM(NumMentions) as gd_nummentions_covid,
                        SUM(NumSources) as gd_numsources_covid,
                        SUM(NumArticles) as gd_numarticles_covid,
                        AVG(AvgTone) as gd_avgtone_covid,
                        AVG(GoldsteinScale) as gd_gtscale_covid
                     FROM gdelt 
                     WHERE covid = true 
                     GROUP BY country_code, `date`
                     """)

In [ ]:
df_gdelt_facts_general = spark.sql("""SELECT country_code, `date`, 
                        COUNT(DISTINCT GLOBALEVENTID) as gd_events_general,
                        SUM(NumMentions) as gd_nummentions_general,
                        SUM(NumSources) as gd_numsources_general,
                        SUM(NumArticles) as gd_numarticles_general,
                        AVG(AvgTone) as gd_avgtone_general,
                        AVG(GoldsteinScale) as gd_gtscale_general
                     FROM gdelt
                     GROUP BY country_code, `date`
                     """)

In [ ]:
df_gdelt_facts = df_gdelt_facts_general.join(df_gdelt_facts_covid,on=['country_code','date'],how="outer")

In [ ]:
df_gdelt_facts = df_gdelt_facts.withColumn('gd_events_covid_perc', col('gd_events_covid') / col('gd_events_general'))
df_gdelt_facts = df_gdelt_facts.withColumn('gd_nummentions_covid_perc', col('gd_nummentions_covid') / col('gd_nummentions_general'))
df_gdelt_facts = df_gdelt_facts.withColumn('gd_numsources_covid_perc', col('gd_numsources_covid') / col('gd_numsources_general'))
df_gdelt_facts = df_gdelt_facts.withColumn('gd_numarticles_covid_perc', col('gd_numarticles_covid') / col('gd_numarticles_general'))

In [ ]:
df_gdelt_facts = df_gdelt_facts.withColumnRenamed('country_code', 'regionId')

In [ ]:
df_fact_covid = df_fact_covid.join(df_gdelt_facts, on=['regionId','date'], how='left')

In [ ]:
factCount_final = df_fact_covid.count()

In [ ]:
assert factCount == factCount_final

In [ ]:
len(df_fact_covid.columns)

In [ ]:
df_fact_covid.printSchema()

In [ ]:
df_fact_covid.write.mode('overwrite').partitionBy("country_code").parquet(folder_s2+"/factCovid.parquet")